_한국어로 기계번역됨_


# 그래프에 스레드 수준의 지속성을 추가하는 방법

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음 내용을 알고 있다고 가정합니다:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/">
                    지속성
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/memory/">
                    메모리
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#chat-models/">
                    챗 모델
                </a>
            </li>        
        </ul>
    </p>
</div> 

많은 AI 애플리케이션은 여러 상호작용 간에 맥락을 공유하기 위해 메모리가 필요합니다. LangGraph에서는 이러한 유형의 메모리를 [스레드 수준의 지속성](https://langchain-ai.github.io/langgraph/concepts/persistence)으로 모든 [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.StateGraph)에 추가할 수 있습니다.

LangGraph 그래프를 생성할 때, 그래프를 컴파일할 때 [체크포인터](https://langchain-ai.github.io/langgraph/reference/checkpoints/#basecheckpointsaver)를 추가하여 상태를 지속하도록 설정할 수 있습니다:

```python
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()
graph.compile(checkpointer=checkpointer)
```

이 가이드는 그래프에 스레드 수준의 지속성을 추가하는 방법을 보여줍니다.

<div class="admonition tip">
    <p class="admonition-title">참고</p>
    <p>
        여러 대화나 사용자 간에 <b>공유</b>되는 메모리가 필요하다면 (크로스 스레드 지속성), 이 <a href="https://langchain-ai.github.io/langgraph/how-tos/cross-thread-persistence/">하우투 가이드</a>를 확인하세요.
    </p>
</div>


## 설정

먼저 필요한 패키지를 설치해야 합니다.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic


다음으로, 사용할 LLM인 Anthropic의 API 키를 설정해야 합니다.


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")


ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 파악하고 성능을 향상시키세요. LangSmith를 사용하면 LangGraph로 구축된 LLM 앱을 디버그, 테스트 및 모니터링하는 데 필요한 추적 데이터를 활용할 수 있습니다. 사용을 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요. 
    </p>
</div>


## 그래프 정의

우리는 [챗 모델](https://python.langchain.com/docs/concepts/#chat-models)을 호출하는 단일 노드 그래프를 사용할 것입니다.

먼저 우리가 사용할 모델을 정의하겠습니다:


In [3]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")


이제 `StateGraph`를 정의하고 모델 호출 노드를 추가할 수 있습니다:


In [4]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, MessagesState, START


def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
graph = builder.compile()


이 그래프를 사용하려고 하면 대화의 맥락이 상호작용 간에 유지되지 않을 것입니다.


In [5]:
input_message = {"role": "user", "content": "hi! I'm bob"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

input_message = {"role": "user", "content": "what's my name?"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello Bob! It's nice to meet you. How are you doing today? Is there anything I can help you with or would you like to chat about something in particular?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I apologize, but I don't have access to your personal information, including your name. I'm an AI language model designed to provide general information and answer questions to the best of my ability based on my training data. I don't have any information about individual users or their personal details. If you'd like to share your name, you're welcome to do so, but I won't be able to recall it in future conversations.


## 지속성 추가

지속성을 추가하려면 그래프를 컴파일할 때 [체크포인터](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.base.BaseCheckpointSaver)를 전달해야 합니다.


In [6]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
# If you're using LangGraph Cloud or LangGraph Studio, you don't need to pass the checkpointer when compiling the graph, since it's done automatically.


<div class="admonition tip">
    <p class="admonition-title">참고</p>
    <p>
        LangGraph Cloud 또는 LangGraph Studio를 사용하고 있다면, 그래프를 컴파일할 때 체크포인터를 전달할 필요가 <strong>없습니다</strong>, 이유는 자동으로 처리되기 때문입니다.
    </p>
</div>


이제 에이전트와 상호작용할 수 있으며, 이전 메시지를 기억하고 있음을 알 수 있습니다!


In [7]:
config = {"configurable": {"thread_id": "1"}}
input_message = {"role": "user", "content": "hi! I'm bob"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello Bob! It's nice to meet you. How are you doing today? Is there anything in particular you'd like to chat about or any questions you have that I can help you with?


이전 대화를 항상 재개할 수 있습니다:


In [8]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob, as you introduced yourself at the beginning of our conversation.


새로운 대화를 시작하고 싶다면, 다른 `thread_id`를 전달하면 됩니다. 뿅! 모든 기억이 사라집니다!


In [9]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in graph.stream(
    {"messages": [input_message]},
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what's is my name?
================================== Ai Message ==================================

I apologize, but I don't have access to your personal information, including your name. As an AI language model, I don't have any information about individual users unless it's provided within the conversation. If you'd like to share your name, you're welcome to do so, but otherwise, I won't be able to know or guess it.
